In [1]:
import cello_multiplier as cm
from onto_lib import general_ontology_tools as got
import pandas as pd

Unable to import pygraphviz. Visualization is disabled.
Loading ontology from C:\Users\julia\anaconda3\lib\site-packages\onto_lib\obo\DOID.17-01-30.obo ...
Loading ontology from C:\Users\julia\anaconda3\lib\site-packages\onto_lib\obo\UBERON.17-01-30.obo ...
Loading ontology from C:\Users\julia\anaconda3\lib\site-packages\onto_lib\obo\CL.18-11-13.obo ...
Loading ontology from C:\Users\julia\anaconda3\lib\site-packages\onto_lib\obo\CVCL.17-01-30.obo ...
Loading ontology from C:\Users\julia\anaconda3\lib\site-packages\onto_lib\obo\EFO.17-01-30.obo ...


In [2]:
B_df, Z_df, labels, per_gene_mean, per_gene_std, train_dummies_specific, train_dummies_full = cm.get_default_mats()

In [3]:
# create list of samples by celltypes
from sklearn.preprocessing import MultiLabelBinarizer

sample2types = {
            sample: list(map(got.get_term_name, types_ids))
            for sample, types_ids in labels.items()
        }

types_per_b_samples = B_df.index.map(sample2types).values

mlb = MultiLabelBinarizer()
types_per_b_samples = B_df.index.map(sample2types).values

samples_dummies = pd.DataFrame(mlb.fit_transform(types_per_b_samples), columns=mlb.classes_, index=B_df.index)

celltypes = samples_dummies.columns.tolist()

type2samples = {
    type_: samples_dummies.index[samples_dummies[type_] == 1].tolist()
    for type_ in celltypes
}

In [4]:
# create target list for given cell type with 1 being that cell type and 0 being any other cell type
def set_target_full(celltype, type2samples, fulldf, oversample, max_neg_pos_ratio):
    samplelist = type2samples[celltype]
    
    
    if(oversample == True and len(fulldf) > 0):
        len_negative_train = len(fulldf)-len(samplelist)
        len_positive_train = len(samplelist)
        
        neg_pos_ratio = len_negative_train / len_positive_train
        
        # limit the max oversampling ratio
        neg_pos_ratio = min(neg_pos_ratio, max_neg_pos_ratio)
        
        if(neg_pos_ratio > 1):
            df_train = fulldf.loc[samplelist].sample(
                    n=int((neg_pos_ratio - 1) * len_positive_train),
                    replace=True, random_state=111
                )
            fulldf = fulldf.append(df_train)
    
    target_train = pd.Series(0,index = fulldf.index)
    target_train.loc[samplelist] = 1
    
    return(target_train.values, fulldf)

In [5]:
# classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import precision_score, recall_score
import numpy as np
import warnings
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

In [6]:
#types_with_data = types_sizes[types_sizes > 50].index.values
all_types = samples_dummies.columns

In [7]:

classifiers ={}

successful_celltypes = []
unsuccessful_celltypes = []

for cell_type in tqdm(all_types):
        train_target, train_data = set_target_full(
            cell_type, type2samples,B_df, oversample=True,
            max_neg_pos_ratio=2
        )

        if(1 in train_target  and 0 in train_target ):
            # lasso penalty
            classifiers[cell_type] = LogisticRegression(solver = "liblinear",penalty = "l1",random_state=111 )

            classifiers[cell_type].fit(train_data, train_target)
           
        else:
            unsuccessful_celltypes += [cell_type]


   


In [8]:
unsuccessful_celltypes

['cell', 'eukaryotic cell', 'native cell']

In [9]:
import pickle

with open('classifiers.pickle', 'wb') as handle:
    pickle.dump(classifiers, handle, protocol=pickle.HIGHEST_PROTOCOL)